<a href="https://colab.research.google.com/github/lucia1299/DBMS_InvestigativeJournalism/blob/main/RelationLinking(unlabelled_dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#  Processing WN_test.tsv without label

In [ ]:
def process_tsv(file_path, max_tokens=1000):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    sentences = []
    current_sentence = []
    sentence_enders = ".!?"

    for line in lines:
        token = line.strip()
        if token:
            current_sentence.append(token)

            if token in sentence_enders:
                sentences.append(current_sentence)
                current_sentence = []
        else:
            if current_sentence:
                sentences.append(current_sentence)
                current_sentence = []

    if current_sentence:
        sentences.append(current_sentence)

    return sentences

file_path = '/content/drive/MyDrive/Colab Notebooks/TESI/github/WN_test_nolabel.tsv'
sentences = process_tsv(file_path)

In [ ]:
titles = []
title_count = 0

for sentence in sentences:
    if sentence[-1] not in ".;?!:":
        titles.append(sentence)
        title_count += 1

print(f"Number of sentences appended to titles: {title_count}")

#dovrebbe essere il numero degli articoli

Number of sentences appended to titles: 290


In [ ]:
def create_dataset_NOlabel(sentences):
    dataset_list = []
    counterId = 0
    titles = []

    for sentence in sentences:
        if sentence[-1] not in ".;?!:":
            titles.append(sentence)

    for i, sentence in enumerate(sentences):
        if sentence[-1] not in ".;?!:":
            articles_dict = {}
            articles_dict['id'] = counterId
            articles_dict['title'] = sentence
            articles_dict['article'] = ''

            for j in range(i + 1, len(sentences)):
                if sentences[j] in titles:
                    break
                articles_dict['article'] += ' '.join(sentences[j]) + ' '

            dataset_list.append(articles_dict)
            counterId += 1

    return dataset_list

In [ ]:
dataset_list = create_dataset_NOlabel(sentences)

In [ ]:
dataset_list[1]

{'id': 1,
 'title': ['Rivolta',
  'del',
  'pane',
  'nel',
  'Magreb',
  ',',
  '50',
  'morti',
  'in',
  'Tunisia'],
 'article': '50 morti in Tunisia , scontri in Algeria e tensioni in Marocco , aumenti di prezzo per grano , olio , zucchero , oltre che del petrolio . La sommossa ricorda quella dell\' assalto al forno delle grucce , di manzoniana memoria . È di nuovo la " rivolta del pane " , in tutto il Magreb . C\' è chi ricorda che ciclicamente si ripetono le stesse scene , c\' è chi invece vede l\' inizio di un periodo sempre più negativo sul problema alimentare mondiale . I paesi produttori ed esportatori di grano , come la Francia , stanno assumendo un atteggiamento sempre più prudente nel promettere la consegna di partite di cereali , riaffermando la necessità di ricostruire gli stock . I prezzi nel frattempo da 100 euro la tonnellata sono già saliti al doppio e c\' è anche chi prospetta la possibilità di ulteriori rialzi . Ma sulle cause del fenomeno c\' è ancora incertezza .

# REBEL SECTION

In [ ]:
!pip install transformers wikidata requests rdflib SPARQLWrapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 612.4 kB/s eta 0:00:00


In [ ]:
!pip install nltk

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch
import IPython
from wikidata.client import Client
import requests
import rdflib
from SPARQLWrapper import SPARQLWrapper, JSON

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/mrebel-large", src_lang="it_IT", tgt_lang="tp_XX")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/307 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/mrebel-large")

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

In [ ]:
def extract_relations_from_model_output(text):
    relations = []
    relation = ''
    subject, relation, object_, object_type, subject_type = '','','','',''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").replace("tp_XX", "").replace("__en__", "").split():
        if token == "<triplet>"  or token == "<relation>":
            current = 't'
            if relation != '':
                relations.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
                relation = ''
            subject = ''
        elif token.startswith("<") and token.endswith(">"):
            if current == 't' or current == 'o':
                current = 's'
                if relation != '':
                    relations.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
                object_ = ''
                subject_type = token[1:-1]
            else:
                current = 'o'
                object_type = token[1:-1]
                relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '' and object_type != '' and subject_type != '':
        relations.append({'head': subject.strip(), 'head_type': subject_type, 'type': relation.strip(),'tail': object_.strip(), 'tail_type': object_type})
    return relations

In [ ]:
def from_text_to_kb(text, span_length=128, verbose=False):
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) /
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": 3,
        "forced_bos_token_id": None,
    }
    generated_tokens = model.generate(
        inputs["input_ids"].to(model.device),
        attention_mask=inputs["attention_mask"].to(model.device),
        decoder_start_token_id = tokenizer.convert_tokens_to_ids("tp_XX"),
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // 3
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                "spans": [spans_boundaries[current_span_index]]
            }
            kb.add_relation(relation)
        i += 1

    return kb

In [ ]:
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
from wikidata.client import Client
import ast

class KB():
    def __init__(self):
        self.entities = {}
        self.relations = []

    def are_relations_equal(self, r1, r2):
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def merge_relations(self, r1):
        r2 = [r for r in self.relations if self.are_relations_equal(r1, r)][0]
        spans_to_add = [span for span in r1["meta"]["spans"] if span not in r2["meta"]["spans"]]
        r2["meta"]["spans"] += spans_to_add

    def get_wikidata_data(self, candidate_entity, language='it'):
        client = Client()
        search_url = "https://www.wikidata.org/w/api.php"
        params = {
            'action': 'wbsearchentities',
            'format': 'json',
            'language': language,
            'search': candidate_entity,
            'limit': 1
        }

        try:
            response = requests.get(search_url, params=params)
            search_results = response.json()

            if not search_results.get('search'):
                return None

            entity_id = search_results['search'][0]['id']
            wikidata_entity = client.get(entity_id, load=True)
            title = wikidata_entity.label.get(language, wikidata_entity.label.get('en'))

            entity_data = {
                "title": title,
                "url": f"https://www.wikidata.org/wiki/{entity_id}",
            }
            return entity_data
        except Exception as e:
            print(f"Error fetching data: {e}")
            return None

    def add_entity(self, e):
        self.entities[e["title"]] = {k: v for k, v in e.items() if k != "title"}

    def add_relation(self, r):
        candidate_entities = [r["head"], r["tail"]]
        entities = [self.get_wikidata_data(ent) for ent in candidate_entities]

        for i, e in enumerate(entities):
            if e is None:
                entities[i] = {"title": candidate_entities[i]}
            else:
                self.add_entity(e)

        r["head"] = entities[0]["title"]
        r["tail"] = entities[1]["title"]

        if not self.exists_relation(r):
            self.relations.append(r)
        else:
            self.merge_relations(r)

    class SemanticTripleConverter:
        def __init__(self, entities, relations):
            self.entities = entities
            self.relations = relations
            self.sparql_endpoint = "https://dbpedia.org/sparql"
            self.ontology_predicates = {'is': 'http://www.w3.org/2002/07/owl/sameAs'}

        def get_valid_predicate(self, predicate):

            if predicate in self.ontology_predicates:
                return self.ontology_predicates[predicate]

            external_predicate = self.query_external_ontology(predicate)
            if external_predicate:
                self.ontology_predicates[predicate] = external_predicate
                return external_predicate
            return predicate

        def search_lov_property(self, prop_name):
            base_url = "https://lov.linkeddata.es/dataset/lov/api/v2/term/search"
            params = {"q": prop_name, "type": "property"}

            response = requests.get(base_url, params=params)
            if response.status_code == 200:
                return response.json()
            else:
                return None

        def get_lov_property(self, property_name):
            results = self.search_lov_property(property_name)
            if results:
                max_score = float('-inf')
                result_with_max_score = None

                for result in results.get("results", []):
                    score = result.get('score', float('-inf'))

                    if score > max_score:
                        max_score = score
                        result_with_max_score = result

                if result_with_max_score:
                    return str(result_with_max_score.get('uri', 'N/A'))
            return None

        def query_external_ontology(self, predicate):

            sparql = SPARQLWrapper(self.sparql_endpoint)
            query = f"""
            SELECT ?property ?label
            WHERE {{
                ?property a rdf:Property .
                ?property rdfs:label ?label .
                FILTER (lang(?label) = "en" && CONTAINS(LCASE(?label), LCASE("{predicate}")))
            }}
            """
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            results = sparql.query().convert()

            if results["results"]["bindings"]:
                return results["results"]["bindings"][0]["property"]["value"]
            else:
                return self.get_lov_property(predicate)

    def ensure_string(self, value):
        # Check if value is a string representation of a list
        if isinstance(value, str) and value.startswith("[") and value.endswith("]"):
            try:
                # Safely parse the string as a list using ast.literal_eval
                parsed_value = ast.literal_eval(value)
                # Check if it's a list and has one element, then return the element as a string
                if isinstance(parsed_value, list) and len(parsed_value) == 1:
                    return str(parsed_value[0])
            except (ValueError, SyntaxError):
                # Handle cases where the parsing fails
                pass
        # Default case, convert the value to string
        return str(value)

    def convertTriples(self):
        converter = self.SemanticTripleConverter(self.entities, self.relations)
        triples = []

        for entity in self.entities.keys():
            triple = {
                'subject': entity,
                'predicate': converter.get_valid_predicate('is'),
                'object': self.entities[entity]['url']
            }
            triples.append(triple)

        for relation in self.relations:
            value = converter.get_valid_predicate(relation['type'])
            triple = {
                'subject': relation['head'],
                'predicate': self.ensure_string(value),
                'object': relation['tail']
            }
            triples.append(triple)

        return triples

# Tryout on one article

In [ ]:
# This first tryout demonstrates that after a few triples the model confuses itself when working on a single article

first_try = dataset_list[1]
text = first_try['article']
kb = from_text_to_kb(text)
final_triples = kb.convertTriples()
final_triples

[{'subject': 'rivolta del pane',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q5465517'},
 {'subject': 'Maghreb',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q28227'},
 {'subject': 'Tunisia',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q948'},
 {'subject': 'Marocco',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q1028'},
 {'subject': 'Francia',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q142'},
 {'subject': 'cereale',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q12117'},
 {'subject': 'grano',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q15645384'},
 {'subject': 'Amartya Sen',
  'predicate': 'http://www.w3.org/2002/07

In [ ]:
first_try = dataset_list[1]
text = first_try['article']
print(type(text))

<class 'str'>


Tryout on a group of sentences, instead of the full article

In [ ]:
first_try = dataset_list[1]
text = first_try['article']

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(text, language='italian')

grouped_sentences = [' '.join(sentences[i:i+3]) for i in range(0, len(sentences), 3)]
print(type(grouped_sentences))
grouped_sentences[0]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<class 'list'>


'50 morti in Tunisia , scontri in Algeria e tensioni in Marocco , aumenti di prezzo per grano , olio , zucchero , oltre che del petrolio . La sommossa ricorda quella dell\' assalto al forno delle grucce , di manzoniana memoria . È di nuovo la " rivolta del pane " , in tutto il Magreb .'

In [ ]:
first_try = dataset_list[1]
text = first_try['article']
sentences = sent_tokenize(text, language='italian')
grouped_sentences = [' '.join(sentences[i:i+3]) for i in range(0, len(sentences), 3)]
example = grouped_sentences[0]
kb = from_text_to_kb(example)
final_triples = kb.convertTriples()
final_triples

[{'subject': 'rivolta del pane',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q5465517'},
 {'subject': 'Maghreb',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q28227'},
 {'subject': 'Tunisia',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q948'},
 {'subject': 'Marocco',
  'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
  'object': 'https://www.wikidata.org/wiki/Q1028'},
 {'subject': 'rivolta del pane',
  'predicate': 'http://dbpedia.org/ontology/location',
  'object': 'Maghreb'},
 {'subject': 'Tunisia',
  'predicate': 'http://purl.org/dc/elements/1.1/relation',
  'object': 'Marocco'},
 {'subject': 'Marocco',
  'predicate': 'http://purl.org/dc/elements/1.1/relation',
  'object': 'Tunisia'},
 {'subject': '50 morti in Tunisia',
  'predicate': 'http://dbpedia.org/property/shipTheCountry',
  'object': 'Tunisia'}]

In [ ]:
print(type(final_triples))

<class 'list'>


Tryout on a full article with the method that groups sentences to ease the process

In [ ]:
triples = []

article_dict = dataset_list[1]
text = article_dict['article']
sentences = sent_tokenize(text, language='italian')
grouped_sentences = [' '.join(sentences[i:i+3]) for i in range(0, len(sentences), 3)]
for j in range(len(grouped_sentences)):
    little_group = grouped_sentences[j]
    kb = from_text_to_kb(little_group)
    article_triples = kb.convertTriples()
    triples.append(article_triples)

triples

[[{'subject': 'rivolta del pane',
   'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
   'object': 'https://www.wikidata.org/wiki/Q5465517'},
  {'subject': 'Maghreb',
   'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
   'object': 'https://www.wikidata.org/wiki/Q28227'},
  {'subject': 'Tunisia',
   'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
   'object': 'https://www.wikidata.org/wiki/Q948'},
  {'subject': 'Marocco',
   'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
   'object': 'https://www.wikidata.org/wiki/Q1028'},
  {'subject': 'rivolta del pane',
   'predicate': 'http://dbpedia.org/ontology/location',
   'object': 'Maghreb'},
  {'subject': 'Tunisia',
   'predicate': 'http://purl.org/dc/elements/1.1/relation',
   'object': 'Marocco'},
  {'subject': 'Marocco',
   'predicate': 'http://purl.org/dc/elements/1.1/relation',
   'object': 'Tunisia'},
  {'subject': '50 morti in Tunisia',
   'predicate': 'http://dbpedia.org/property/shipTheCountry',
   'object': 'Tuni

# Create triples for two example articles

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def create_triples_example(dataset_example):
    triples = []

    article_dict = dataset_example
    text = article_dict['article']
    sentences = sent_tokenize(text, language='italian')
    grouped_sentences = [' '.join(sentences[i:i+3]) for i in range(0, len(sentences), 3)]
    for j in range(len(grouped_sentences)):
        little_group = grouped_sentences[j]
        kb = from_text_to_kb(little_group)
        article_triples = kb.convertTriples()
        triples.append(article_triples)

    return triples

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
example_triples = create_triples_example(dataset_list[1]) + create_triples_example(dataset_list[5])

In [ ]:
import json

file_path = '/content/drive/MyDrive/Colab Notebooks/TESI/github/example_triples_nolabel.json'

# Write to the JSON file in Google Drive
with open(file_path, 'w') as file:
    json.dump(example_triples, file, indent=4)

Creating triples for the whole dataset - TO BE RUN (BUT IT TAKES TOO MUCH TIME!)

In [ ]:
def generate_triples_labelled(dataset_list):

    triples = []

    for i in range(len(dataset_list)):
        article_dict = dataset_list[i]
        text = article_dict['article']
        sentences = sent_tokenize(text, language='italian')
        grouped_sentences = [' '.join(sentences[i:i+3]) for i in range(0, len(sentences), 3)]
        for j in range(len(grouped_sentences)):
            little_group = grouped_sentences[j]
            kb = from_text_to_kb(little_group)
            article_triples = kb.convertTriples()
            triples.append(article_triples)

    return triples

In [ ]:
triples = generate_triples_labelled(dataset_list)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-db8ccafb56bb>", line 1, in <cell line: 1>
    triples = generate_triples_labelled(dataset_list)
  File "<ipython-input-23-58f64399197b>", line 12, in generate_triples_labelled
    kb = from_text_to_kb(little_group)
  File "<ipython-input-12-224df637e982>", line 61, in from_text_to_kb
    kb.add_relation(relation)
  File "<ipython-input-13-e98ff237c9ee>", line 58, in add_relation
    entities = [self.get_wikidata_data(ent) for ent in candidate_entities]
  File "<ipython-input-13-e98ff237c9ee>", line 58, in <listcomp>
    entities = [self.get_wikidata_data(ent) for ent in candidate_entities]
  File "<ipython-input-13-e98ff237c9ee>", line 41, in get_wikidata_data
    wikidata_entity = client.get(entity_id, load=True)
  File "/usr/local/lib/python3.10/dist-packages/wikidat

TypeError: object of type 'NoneType' has no len()